In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score, make_scorer, classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/spambase.data'

try:
  df = pd.read_csv(file_path, header=None)
  print("Data loaded successfully.")
  print("Shape of the dataset:", df.shape)
  df.head()

except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
except Exception as e:
  print(f"An error occurred: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data loaded successfully.
Shape of the dataset: (4601, 58)


In [3]:
column_list = ["word_freq_make",
"word_freq_address",
"word_freq_all",
"word_freq_3d",
"word_freq_our",
"word_freq_over",
"word_freq_remove",
"word_freq_internet",
"word_freq_order",
"word_freq_mail",
"word_freq_receive",
"word_freq_will",
"word_freq_people",
"word_freq_report",
"word_freq_addresses",
"word_freq_free",
"word_freq_business",
"word_freq_email",
"word_freq_you",
"word_freq_credit",
"word_freq_your",
"word_freq_font",
"word_freq_000",
"word_freq_money",
"word_freq_hp",
"word_freq_hpl",
"word_freq_george",
"word_freq_650",
"word_freq_lab",
"word_freq_labs",
"word_freq_telnet",
"word_freq_857",
"word_freq_data",
"word_freq_415",
"word_freq_85",
"word_freq_technology",
"word_freq_1999",
"word_freq_parts",
"word_freq_pm",
"word_freq_direct",
"word_freq_cs",
"word_freq_meeting",
"word_freq_original",
"word_freq_project",
"word_freq_re",
"word_freq_edu",
"word_freq_table",
"word_freq_conference",
"char_freq_;",
"char_freq_(",
"char_freq_[",
"char_freq_!",
"char_freq_$",
"char_freq_#",
"capital_run_length_average",
"capital_run_length_longest",
"capital_run_length_total",
"spam"
]

In [4]:
df.columns = column_list
df

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [5]:
df.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [6]:
print(df.isnull().any())


word_freq_make                False
word_freq_address             False
word_freq_all                 False
word_freq_3d                  False
word_freq_our                 False
word_freq_over                False
word_freq_remove              False
word_freq_internet            False
word_freq_order               False
word_freq_mail                False
word_freq_receive             False
word_freq_will                False
word_freq_people              False
word_freq_report              False
word_freq_addresses           False
word_freq_free                False
word_freq_business            False
word_freq_email               False
word_freq_you                 False
word_freq_credit              False
word_freq_your                False
word_freq_font                False
word_freq_000                 False
word_freq_money               False
word_freq_hp                  False
word_freq_hpl                 False
word_freq_george              False
word_freq_650               

In [7]:
df['spam'].value_counts()

,count
spam,
0,2788
1,1813


## (A)

In [8]:
from sklearn.model_selection import train_test_split

X_features = df[df.columns[:-1]]
y_feature = df['spam']
X_CV, X_test, y_CV, y_test = train_test_split(
    X_features, y_feature, test_size=0.20, random_state=60
)

In [9]:
x_scaler = StandardScaler()
X_CV_scaled = x_scaler.fit_transform(X_CV)
X_test_scaled = x_scaler.transform(X_test)

In [14]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "KNN": KNeighborsClassifier(),
    "DecisionTree": DecisionTreeClassifier(class_weight='balanced'),
    "SVC": SVC(class_weight='balanced', probability=True),
    "NeuralNet": MLPClassifier(max_iter=300, early_stopping=True, random_state=42),
    "LightGBM": LGBMClassifier(class_weight='balanced', random_state=42, verbose=-1)
}

param_grids = {
    "LogisticRegression": {
        "model__C": [0.01, 0.1, 1, 10]
    },
    "KNN": {
        "model__n_neighbors": [3, 5, 7],
        "model__weights": ["uniform", "distance"]
    },
    "DecisionTree": {
        "model__max_depth": [4, 6, 10, None],
        "model__min_samples_split": [2, 5, 10]
    },
    "SVC": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["linear", "rbf"]
    },
    "NeuralNet": {
        "model__hidden_layer_sizes": [(50,), (100,)],
        "model__alpha": [0.0001, 0.001],
        "model__solver": ["adam"],
        "model__activation": ["relu"]
    },
    "LightGBM": {
        "model__n_estimators": [50, 100],
        "model__max_depth": [3, 5, 7],
        "model__learning_rate": [0.05, 0.1]
    }
}

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
accuracy_scorer = make_scorer(accuracy_score)

results = {}
roc_data = {}

for model_name, model in models.items():
    print(f"Running Nested CV for {model_name}...")
    nested_scores = []
    all_y_true = []
    all_y_scores = []

    for train_idx, valid_idx in outer_cv.split(X_CV_scaled, y_CV):
        X_train, X_valid = X_CV_scaled[train_idx], X_CV_scaled[valid_idx]
        y_train, y_valid = y_CV.iloc[train_idx], y_CV.iloc[valid_idx]

        pipeline = Pipeline([("scaler", StandardScaler()), ("model", model)])
        param_grid = param_grids[model_name]

        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=accuracy_scorer,
            cv=inner_cv,
            refit=True,
            n_jobs=-1
        )

        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        y_pred = best_model.predict(X_valid)
        y_score = best_model.predict_proba(X_valid)[:, 1] if hasattr(best_model, "predict_proba") else best_model.decision_function(X_valid)

        acc = accuracy_score(y_valid, y_pred)
        nested_scores.append(acc)
        all_y_true.extend(y_valid)
        all_y_scores.extend(y_score)

    mean_acc = np.mean(nested_scores)
    std_acc = np.std(nested_scores)
    results[model_name] = {"mean_accuracy": mean_acc, "std_accuracy": std_acc}
    print(f"{model_name} - Mean Accuracy: {mean_acc:.4f}, Std: {std_acc:.4f}")

    fpr, tpr, _ = roc_curve(all_y_true, all_y_scores)
    roc_auc = auc(fpr, tpr)
    roc_data[model_name] = (fpr, tpr, roc_auc)


for model_name, (fpr, tpr, roc_auc) in roc_data.items():
    results[model_name]["auc"] = roc_auc

# Create DataFrame and display results
results_df = pd.DataFrame(results).T
results_df = results_df[["mean_accuracy", "std_accuracy", "auc"]]  # Arrange columns

print("\nNested CV Accuracy & AUC Results:")
print(results_df)


Running Nested CV for LogisticRegression...
LogisticRegression - Mean Accuracy: 0.9291, Std: 0.0122
Running Nested CV for KNN...
KNN - Mean Accuracy: 0.9111, Std: 0.0085
Running Nested CV for DecisionTree...
DecisionTree - Mean Accuracy: 0.9076, Std: 0.0064
Running Nested CV for SVC...
SVC - Mean Accuracy: 0.9310, Std: 0.0107
Running Nested CV for NeuralNet...
NeuralNet - Mean Accuracy: 0.9353, Std: 0.0106
Running Nested CV for LightGBM...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

LightGBM - Mean Accuracy: 0.9538, Std: 0.0052

Nested CV Accuracy & AUC Results:
                    mean_accuracy  std_accuracy       auc
LogisticRegression       0.929076      0.012195  0.970816
KNN                      0.911141      0.008489  0.961447
DecisionTree             0.907609      0.006373  0.921576
SVC                      0.930978      0.010747  0.971670
NeuralNet                0.935326      0.010580  0.976483
LightGBM                 0.953804      0.005227  0.988371


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Among all the models evaluated using nested cross-validation, LightGBM demonstrated the highest predictive performance, achieving the highest mean classification accuracy (0.9538) with the lowest standard deviation (0.0052). This indicates that LightGBM not only made the most accurate predictions on average but also produced the most consistent results across different folds. Its performance surpassed other models like logistic regression, neural networks, and support vector machines, making it the most reliable choice for the spam classification task in terms of overall accuracy and robustness.

In [ ]:
param_grid = {
    "n_estimators": [50, 100],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 5, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=LGBMClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_CV_scaled, y_CV)

best_model = grid_search.best_estimator_
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate on test set
y_pred = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Set Accuracy: {accuracy:.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 1453, number of negative: 2227
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002464 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6976
[LightGBM] [Info] Number of data points in the train set: 3680, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.394837 -> initscore=-0.427025
[LightGBM] [Info] Start training from score -0.427025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


The final tuned LightGBM model achieved a test set accuracy of 95.01%, demonstrating strong overall predictive performance. The confusion matrix shows that the model correctly identified 543 non-spam emails and 332 spam emails, while misclassifying only 18 non-spam as spam and 28 spam as non-spam. The classification report further highlights a balanced precision and recall across both classes, with a macro-averaged F1-score of 0.95, indicating the model performs well in both minimizing false positives and false negatives. These results confirm that the model is well-suited for the task of spam detection.

## (B)

In [16]:
def cost_sensitive_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    cost = 10 * fp + fn
    return -cost

cost_scorer = make_scorer(cost_sensitive_score, greater_is_better=True)

In [20]:
# Define classification models
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "KNN": KNeighborsClassifier(),
    "DecisionTree": DecisionTreeClassifier(class_weight='balanced'),
    "SVC": SVC(class_weight='balanced', probability=True),
    "NeuralNet": MLPClassifier(max_iter=300, early_stopping=True, random_state=42),
    "LightGBM": LGBMClassifier(class_weight='balanced', random_state=42, verbose=-1)
}

# Define hyperparameter grids
param_grids = {
    "LogisticRegression": {
        "model__C": [0.01, 0.1, 1, 10]
    },
    "KNN": {
        "model__n_neighbors": [3, 5, 7],
        "model__weights": ["uniform", "distance"]
    },
    "DecisionTree": {
        "model__max_depth": [4, 6, 10, None],
        "model__min_samples_split": [2, 5, 10]
    },
    "SVC": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["linear", "rbf"]
    },
    "NeuralNet": {
        "model__hidden_layer_sizes": [(50,), (100,)],
        "model__alpha": [0.0001, 0.001],
        "model__solver": ["adam"],
        "model__activation": ["relu"]
    },
    "LightGBM": {
        "model__n_estimators": [50, 100],
        "model__max_depth": [3, 5, 7],
        "model__learning_rate": [0.05, 0.1]
    }
}

# CV setup
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Store results
results = {}

# Nested cross-validation with cost-sensitive scoring and AUC
for model_name, model in models.items():
    print(f"Running Nested CV for {model_name}...")
    nested_costs = []
    all_y_true = []
    all_y_scores = []

    for train_idx, valid_idx in outer_cv.split(X_CV_scaled, y_CV):
        X_train, X_valid = X_CV_scaled[train_idx], X_CV_scaled[valid_idx]
        y_train, y_valid = y_CV.iloc[train_idx], y_CV.iloc[valid_idx]

        pipeline = Pipeline([("scaler", StandardScaler()), ("model", model)])
        param_grid = param_grids[model_name]

        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=cost_scorer,
            cv=inner_cv,
            refit=True,
            n_jobs=-1
        )

        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        y_pred = best_model.predict(X_valid)
        cost = -cost_sensitive_score(y_valid, y_pred)
        nested_costs.append(cost)

        # For AUC
        if hasattr(best_model, "predict_proba"):
            y_score = best_model.predict_proba(X_valid)[:, 1]
        else:
            y_score = best_model.decision_function(X_valid)
        all_y_true.extend(y_valid)
        all_y_scores.extend(y_score)

    mean_cost = np.mean(nested_costs)
    std_cost = np.std(nested_costs)
    roc_auc = roc_auc_score(all_y_true, all_y_scores)

    results[model_name] = {
        "mean_cost": mean_cost,
        "std_cost": std_cost,
        "auc": roc_auc
    }
    print(f"{model_name} - Mean Cost: {mean_cost:.2f}, Std: {std_cost:.2f}, AUC: {roc_auc:.4f}")

# Display results
results_df = pd.DataFrame(results).T
print("\nNested CV Cost & AUC Results:")
print(results_df[["mean_cost", "std_cost", "auc"]])

Running Nested CV for LogisticRegression...
LogisticRegression - Mean Cost: 314.20, Std: 66.98, AUC: 0.9702
Running Nested CV for KNN...
KNN - Mean Cost: 310.00, Std: 29.91, AUC: 0.9633
Running Nested CV for DecisionTree...
DecisionTree - Mean Cost: 350.40, Std: 33.60, AUC: 0.9191
Running Nested CV for SVC...
SVC - Mean Cost: 253.40, Std: 40.74, AUC: 0.9755
Running Nested CV for NeuralNet...
NeuralNet - Mean Cost: 237.60, Std: 55.14, AUC: 0.9757
Running Nested CV for LightGBM...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

LightGBM - Mean Cost: 191.00, Std: 35.75, AUC: 0.9879

Nested CV Cost & AUC Results:
                    mean_cost   std_cost       auc
LogisticRegression      314.2  66.978803  0.970171
KNN                     310.0  29.913208  0.963252
DecisionTree            350.4  33.595238  0.919143
SVC                     253.4  40.736225  0.975541
NeuralNet               237.6  55.138371  0.975737
LightGBM                191.0  35.754720  0.987892


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LightGBM was selected as the best-performing model because it achieved the lowest mean misclassification cost (191.0) while also delivering the highest AUC score (0.9879) among all models. This indicates that LightGBM not only minimized costly errors—especially important in a cost-sensitive context—but also excelled at distinguishing between spam and non-spam emails across all decision thresholds.

Now training LightGBM on entire training dataset

In [21]:
param_grid = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.05, 0.1],
    "max_depth": [5, 7, 9],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [0.5, 1, 1.5]
}

grid_search = GridSearchCV(
    estimator=LGBMClassifier(class_weight='balanced', random_state=42),
    param_grid=param_grid,
    scoring=cost_scorer,
    cv=5,
    n_jobs=-1,
    verbose=0
)

grid_search.fit(X_CV_scaled, y_CV)

best_model = grid_search.best_estimator_
print("Best Hyperparameters:", grid_search.best_params_)

y_pred = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Set Accuracy: {accuracy:.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
cost = 10 * fp + fn
print(f"\nMisclassification Cost on Test Set: {cost}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Best Hyperparameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 150, 'reg_alpha': 0.1, 'reg_lambda': 0.5, 'subsample': 0.7}

Test Set Accuracy: 0.9544

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96       561
           1       0.94      0.94      0.94       360

    accuracy                           0.95       921
   macro avg       0.95      0.95      0.95       921
weighted avg       0.95      0.95      0.95       921


Confusion Matrix:
 [[540  21]
 [ 21 339]]

Misclassification Cost on Test Set: 231


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


## Classification Modeling Summary

### Part (A): Optimizing for Accuracy

- Evaluated six models using **nested cross-validation**:
  - Logistic Regression, KNN, Decision Tree, SVC, Neural Network, LightGBM
- Used `accuracy` and `AUC` for evaluation
- **Model comparison from NestedCV:**

| Model              | Mean Accuracy | Std Accuracy | AUC      |
|-------------------|---------------|---------------|----------|
| LogisticRegression| 0.9291        | 0.0122        | 0.9708   |
| KNN               | 0.9111        | 0.0085        | 0.9614   |
| DecisionTree      | 0.9076        | 0.0064        | 0.9216   |
| SVC               | 0.9310        | 0.0107        | 0.9717   |
| NeuralNet         | 0.9353        | 0.0106        | 0.9765   |
| **LightGBM**      | **0.9538**    | **0.0052**    | **0.9884** |


- **Best Model**: **LightGBM**
  - Mean Accuracy: **95.38%**
  - AUC: **0.9884**
- **Trained LightGBM on full training dataset**
  - **Test Accuracy**: 95.01%
  - **Confusion Matrix**:
    ```
    [[543  18]
     [ 28 332]]
    ```

---

### Part (B): Cost-Sensitive Classification (10:1 ratio)

- Evaluated six models using **nested cross-validation**:
  - Logistic Regression, KNN, Decision Tree, SVC, Neural Network, LightGBM
- Models evaluated using a **custom misclassification cost** metric
- Also tracked **AUC** for class separation
- **Model comparison from NestedCV:**

| Model              | Mean Cost | Std Cost | AUC      |
|-------------------|-----------|----------|----------|
| LogisticRegression| 314.2     | 66.98    | 0.9702   |
| KNN               | 310.0     | 29.91    | 0.9633   |
| DecisionTree      | 350.4     | 33.60    | 0.9191   |
| SVC               | 253.4     | 40.74    | 0.9755   |
| NeuralNet         | 237.6     | 55.14    | 0.9757   |
| **LightGBM**      | **191.0** | **35.75**| **0.9879**|

- **Best Model on NestedCV**: **LightGBM**
  - Mean Cost: **191**
  - AUC: **0.9879**
- **Trained LightGBM on full training dataset**
  - **Test Accuracy**: 95.44%
  - **Misclassification Cost**: **231**
  - **Confusion Matrix**:
    ```
    [[540  21]
     [ 21 339]]
    ```

---

### Final Notes

- LightGBM consistently outperformed all other models in both accuracy and cost-sensitive scenarios.
- Neural Net and SVC were strong contenders, but LightGBM provided the best trade-off between accuracy, cost, and AUC.
